In [1]:
import argparse
import torch
import numpy as np
import models.ULIP_models as models
from utils.tokenizer import SimpleTokenizer
from utils.utils import get_model, get_dataset
from data.shapenet_3d import *
import torch.multiprocessing as mp
import gc
mp.set_start_method('spawn', force=True)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
class Args:
    def __init__(self):
        self.ckpt_path = "ckpt/ULIP-2-PointBERT-10k-colored-pc-pretrained.pt"  # Path to the pretrained checkpoint
        self.model = "ULIP2_PointBERT_Colored"  # Model architecture to use
        self.batch_size = 1  # Batch size for inference
        self.num_workers = 4  # Number of workers for data loading
        
args = Args()

In [3]:
def load_model(args):
    """Load the ULIP-2 model from a checkpoint"""
    print(f"Loading model {args.model} from {args.ckpt_path}...")
    ckpt = torch.load(args.ckpt_path, map_location='cpu')
    state_dict = {k.replace('module.', ''): v for k, v in ckpt['state_dict'].items()}

    model = getattr(models, args.model)(args=args)
    model.load_state_dict(state_dict, strict=False)
    model.cuda().eval()
    return model

# Load the model
model = load_model(args)

Loading model ULIP2_PointBERT_Colored from ckpt/ULIP-2-PointBERT-10k-colored-pc-pretrained.pt...
Get openclip model:
Finished loading the openclip model.
training from scratch for pointbert.
model size:
32.50M


In [4]:
def load_point(ply_file, device='cuda'):
    """Load point cloud from a .ply file."""
    verts, _ = pytorch3d.io.load_ply(ply_file)
    assert isinstance(verts, torch.Tensor) and verts.shape[1] == 3, "The point cloud should be 3D"
    
    # Add default RGB values (zeros)
    rgb = torch.zeros_like(verts)
    point_cloud = torch.cat([verts, rgb], dim=1)
    
    # Verify final dimensions and move to device
    assert point_cloud.shape[1] == 6, f"Expected 6 channels (XYZ+RGB), got {point_cloud.shape[1]}"
    
    return point_cloud.to(device)

In [5]:
def extract_features_from_ply(model, ply_file, device='cuda'):
    """Extract features from a single point cloud using the ULIP-2 model."""
    # Load the point cloud
    point_cloud = load_point(ply_file, device=device)
    
    # Reshape the point cloud to have batch dimension
    point_cloud = point_cloud.unsqueeze(0)  # Shape: (1, N, 6)
    
    # Extract features using the model
    with torch.no_grad():
        features = get_model(model).encode_pc(point_cloud)
        features = features / features.norm(dim=-1, keepdim=True)
        
    return features.cpu().numpy()

In [8]:
def load_npy(npy_file):
    """Load a .npy file containing point cloud data."""
    data = np.load(npy_file)
    assert data.ndim == 2 and data.shape[1] == 6, "The .npy file should contain 6D point cloud data"
    return torch.tensor(data, dtype=torch.float32)

def calculate_similarity(features1, features2):
    """Calculate cosine similarity between two feature sets."""
    assert features1.shape == features2.shape, "Feature sets must have the same shape"
    similarity = np.dot(features1, features2.T)
    return similarity

In [6]:
ply_files = ["/workspace/vis_results/SDFusion/results/table/1a35db5eb4f931a3eba0693b2c2a423b/normalize/points_normalize_rotate_90.ply", "/workspace/vis_results/SDFusion/results/table/1a35db5eb4f931a3eba0693b2c2a423b/normalize/points_normalize_rotate_180.ply", "/workspace/vis_results/SDFusion/results/table/1a35db5eb4f931a3eba0693b2c2a423b/normalize/points_normalize_rotate_270.ply", "/workspace/vis_results/SDFusion/results/table/1a35db5eb4f931a3eba0693b2c2a423b/normalize/points_normalize_rotate_360.ply"]

In [7]:
for ply_file in ply_files:
    features = extract_features_from_ply(model, ply_file)
    print(f"Extracted features for {ply_file}: {features.shape}")
    # Save the features to a file
    np.save(ply_file.replace('.ply', '_features.npy'), features)
    print(f"Features saved to {ply_file.replace('.ply', '_features.npy')}")

Extracted features for /workspace/vis_results/SDFusion/results/table/1a35db5eb4f931a3eba0693b2c2a423b/normalize/points_normalize_rotate_90.ply: (1, 1280)
Features saved to /workspace/vis_results/SDFusion/results/table/1a35db5eb4f931a3eba0693b2c2a423b/normalize/points_normalize_rotate_90_features.npy
Extracted features for /workspace/vis_results/SDFusion/results/table/1a35db5eb4f931a3eba0693b2c2a423b/normalize/points_normalize_rotate_180.ply: (1, 1280)
Features saved to /workspace/vis_results/SDFusion/results/table/1a35db5eb4f931a3eba0693b2c2a423b/normalize/points_normalize_rotate_180_features.npy
Extracted features for /workspace/vis_results/SDFusion/results/table/1a35db5eb4f931a3eba0693b2c2a423b/normalize/points_normalize_rotate_270.ply: (1, 1280)
Features saved to /workspace/vis_results/SDFusion/results/table/1a35db5eb4f931a3eba0693b2c2a423b/normalize/points_normalize_rotate_270_features.npy
Extracted features for /workspace/vis_results/SDFusion/results/table/1a35db5eb4f931a3eba0693b

In [9]:
# calculate similarity
features_list = []
for ply_file in ply_files:
    features = np.load(ply_file.replace('.ply', '_features.npy'))
    features_list.append(features)
similarity_matrix = np.zeros((len(features_list), len(features_list)))
for i in range(len(features_list)):
    for j in range(len(features_list)):
        similarity_matrix[i][j] = calculate_similarity(features_list[i], features_list[j])
print("Similarity matrix:")
print(similarity_matrix)


Similarity matrix:
[[0.99999988 0.99390501 0.99378866 0.99406528]
 [0.99390501 1.         0.99393326 0.99459052]
 [0.99378866 0.99393326 1.         0.99542034]
 [0.99406528 0.99459052 0.99542034 0.99999994]]


In [14]:
# Now load all the results files
SDFusion_Results_plys = []
ROOT_DIR = "/workspace/vis_results/SDFusion/results/"
for category in os.listdir(ROOT_DIR):
    category_dir = os.path.join(ROOT_DIR, category)
    if os.path.isdir(category_dir):
        for subdir in os.listdir(category_dir):
            subdir_path = os.path.join(category_dir, subdir, "normalize")
            if os.path.isdir(subdir_path):
                for file in os.listdir(subdir_path):
                    if file.endswith('_normalize.ply'):
                        SDFusion_Results_plys.append(os.path.join(subdir_path, file))
# Save to the npy to same dir of the file

for ply_file in SDFusion_Results_plys:
    features = extract_features_from_ply(model, ply_file)
    print(f"Extracted features for {ply_file}: {features.shape}")
    # Save the features to a file
    np.save(os.path.join(os.path.dirname(ply_file),"ulip_feature.npy"), features)
    print(f"Features saved to {os.path.join(os.path.dirname(ply_file),'ulip_feature.npy')}")


Extracted features for /workspace/vis_results/SDFusion/results/chair/1055f78d441d170c4f3443b22038d340/normalize/points_normalize.ply: (1, 1280)
Features saved to /workspace/vis_results/SDFusion/results/chair/1055f78d441d170c4f3443b22038d340/normalize/ulip_feature.npy
Extracted features for /workspace/vis_results/SDFusion/results/chair/10dc303144fe5d668d1b9a1d97e2846/normalize/points_normalize.ply: (1, 1280)
Features saved to /workspace/vis_results/SDFusion/results/chair/10dc303144fe5d668d1b9a1d97e2846/normalize/ulip_feature.npy
Extracted features for /workspace/vis_results/SDFusion/results/chair/11525a18678f7ce6ae1e1181f20bb9c8/normalize/points_normalize.ply: (1, 1280)
Features saved to /workspace/vis_results/SDFusion/results/chair/11525a18678f7ce6ae1e1181f20bb9c8/normalize/ulip_feature.npy
Extracted features for /workspace/vis_results/SDFusion/results/chair/13076ebf8b5cc457b8d6f69a14683de3/normalize/points_normalize.ply: (1, 1280)
Features saved to /workspace/vis_results/SDFusion/resu

In [11]:
# remove all the .npy files under ROOT_DIR

import shutil
# os.walk(ROOT_DIR)

for root, dirs, files in os.walk(ROOT_DIR):
    for file in files:
        if file.endswith('.npy'):
            os.remove(os.path.join(root, file))
            print(f"Removed {os.path.join(root, file)}")
            

Removed /workspace/vis_results/SDFusion/results/chair/1055f78d441d170c4f3443b22038d340/points_normalize_features.npy
Removed /workspace/vis_results/SDFusion/results/chair/1055f78d441d170c4f3443b22038d340/normalize/points_normalize_features.npy
Removed /workspace/vis_results/SDFusion/results/chair/10dc303144fe5d668d1b9a1d97e2846/points_normalize_features.npy
Removed /workspace/vis_results/SDFusion/results/chair/10dc303144fe5d668d1b9a1d97e2846/normalize/points_normalize_features.npy
Removed /workspace/vis_results/SDFusion/results/chair/11525a18678f7ce6ae1e1181f20bb9c8/points_normalize_features.npy
Removed /workspace/vis_results/SDFusion/results/chair/11525a18678f7ce6ae1e1181f20bb9c8/normalize/points_normalize_features.npy
Removed /workspace/vis_results/SDFusion/results/chair/13076ebf8b5cc457b8d6f69a14683de3/points_normalize_features.npy
Removed /workspace/vis_results/SDFusion/results/chair/13076ebf8b5cc457b8d6f69a14683de3/normalize/points_normalize_features.npy
Removed /workspace/vis_res

In [15]:
# Now load all the results files
Michelangelo_Results_plys = []
ROOT_DIR = "/workspace/vis_results/Michelangelo/results"
for modeldir in os.listdir(ROOT_DIR):
    model_dir = os.path.join(ROOT_DIR, modeldir)
    if os.path.isdir(model_dir):
        normalize_dir = os.path.join(model_dir, "normalize")
        if os.path.isdir(normalize_dir):
            for file in os.listdir(normalize_dir):
                if file.endswith('_normalize.ply'):
                    Michelangelo_Results_plys.append(os.path.join(normalize_dir, file))
# Save to the npy to same dir of the file

for ply_file in Michelangelo_Results_plys:
    features = extract_features_from_ply(model, ply_file)
    print(f"Extracted features for {ply_file}: {features.shape}")
    # Save the features to a file
    np.save(os.path.join(os.path.dirname(ply_file),"ulip_feature.npy"), features)
    print(f"Features saved to {os.path.join(os.path.dirname(ply_file),'ulip_feature.npy')}")


Extracted features for /workspace/vis_results/Michelangelo/results/10f7179292b04787cf335697521a6511/normalize/points_normalize.ply: (1, 1280)
Features saved to /workspace/vis_results/Michelangelo/results/10f7179292b04787cf335697521a6511/normalize/ulip_feature.npy
Extracted features for /workspace/vis_results/Michelangelo/results/1399af537cd5f80723a295070ca176a0/normalize/points_normalize.ply: (1, 1280)
Features saved to /workspace/vis_results/Michelangelo/results/1399af537cd5f80723a295070ca176a0/normalize/ulip_feature.npy
Extracted features for /workspace/vis_results/Michelangelo/results/18943947ce3b8cae8a94168388287ad5/normalize/points_normalize.ply: (1, 1280)
Features saved to /workspace/vis_results/Michelangelo/results/18943947ce3b8cae8a94168388287ad5/normalize/ulip_feature.npy
Extracted features for /workspace/vis_results/Michelangelo/results/1a1a0794670a2114d6bef0ac9b3a5962/normalize/points_normalize.ply: (1, 1280)
Features saved to /workspace/vis_results/Michelangelo/results/1a1a